# Предварительная обработка данных
Приведение всех признаков к числовым и удаление ненужных

## Описание признаков

| Признак        | Описание                                                                                                                                         |
|----------------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| **PassengerId** | Уникальный числовой идентификатор пассажира. Используется для однозначной идентификации каждого пассажира в наборе данных.                        |
| **Survived**    | Флаг, указывающий, выжил ли пассажир. Значения: <br> - `0` = Нет (погиб), <br> - `1` = Да (выжил).                                               |
| **Pclass**      | Класс билета пассажира, который также отражает социально-экономический статус. Значения: <br> - `1` = Первый класс, <br> - `2` = Второй класс, <br> - `3` = Третий класс. |
| **Name**        | Полное имя пассажира, включая титул (например, Mr., Mrs., Miss и т.д.).                                                                          |
| **Sex**         | Пол пассажира. Возможные значения: <br> - `male` = Мужчина, <br> - `female` = Женщина.                                                           |
| **Age**         | Возраст пассажира в годах. Может содержать пропущенные значения (NaN).                                                                           |
| **SibSp**       | Количество братьев/сестер или супругов на борту Титаника.                                                                                        |
| **Parch**       | Количество родителей или детей на борту Титаника.                                                                                                |
| **Ticket**      | Номер билета пассажира. Может быть строковым или числовым значением.                                                                             |
| **Fare**        | Стоимость билета пассажира. Значение может быть дробным числом.                                                                                  |
| **Cabin**       | Номер каюты пассажира. Может содержать пропущенные значения (NaN).                                                                               |
| **Embarked**    | Порт посадки пассажира. Возможные значения: <br> - `C` = Cherbourg, <br> - `Q` = Queenstown, <br> - `S` = Southampton.                           |

## Подключение библиотек

In [221]:
import pandas as pd

## Обработка

In [222]:
datasetType = 'test' 
dfOrignal = pd.read_csv('{}.csv'.format(datasetType))
df = dfOrignal.copy()
df.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


### Удаляем данные, которые никак не повлияют на таргетное значение

In [223]:
df.drop(['PassengerId', 'Name'], axis=1, inplace=True)
df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,34.5,0,0,330911,7.8292,NaN,Q
1,3,female,47.0,1,0,363272,7.0000,NaN,S
2,2,male,62.0,0,0,240276,9.6875,NaN,Q


### Вычисление процентного соотношения пропущенных и непропущенных значений

In [224]:
missingPercent = df.isnull().mean() * 100
nonMisssingPercent = (1 - df.isnull().mean()) * 100

missingStats = pd.DataFrame({
    "Total": df.isnull().sum(), 
    "Percent Missing": missingPercent.round(2),  
    "Percent Non-Missing": nonMisssingPercent.round(2) 
})

print(missingStats)

          Total  Percent Missing  Percent Non-Missing
Pclass        0             0.00               100.00
Sex           0             0.00               100.00
Age          86            20.57                79.43
SibSp         0             0.00               100.00
Parch         0             0.00               100.00
Ticket        0             0.00               100.00
Fare          1             0.24                99.76
Cabin       327            78.23                21.77
Embarked      0             0.00               100.00


### Удаление столбцов с более чем 50% пропущенных значений

In [225]:
columnsToDrop = missingPercent[missingPercent > 50].index
df.drop(columnsToDrop, axis=1, inplace=True)
df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,male,34.5,0,0,330911,7.8292,Q
1,3,female,47.0,1,0,363272,7.0000,S
2,2,male,62.0,0,0,240276,9.6875,Q


### Преобразование категориальных признаков в вещественные значения


In [226]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,0,34.5,0,0,330911,7.8292,1
1,3,1,47.0,1,0,363272,7.0000,2
2,2,0,62.0,0,0,240276,9.6875,1


### Удаляем признак Ticket, так как непонятно по какому правилу преобразовать его в число

In [227]:
df.drop(['Ticket'], axis=1, inplace=True)
df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,1
1,3,1,47.0,1,0,7.0000,2
2,2,0,62.0,0,0,9.6875,1


In [228]:
df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
df['Embarked'].fillna(df['Embarked'].mode()[0])
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,1
1,3,1,47.0,1,0,7.0000,2
2,2,0,62.0,0,0,9.6875,1
3,3,0,27.0,0,0,8.6625,2
4,3,1,22.0,1,1,12.2875,2
...,...,...,...,...,...,...,...
413,3,0,24.0,0,0,8.0500,2
414,1,1,39.0,0,0,108.9000,0
415,3,0,38.5,0,0,7.2500,2
416,3,0,24.0,0,0,8.0500,2


### Сохранение обработанного датасета

In [229]:
df.to_csv('preprocessed_{}.csv'.format(datasetType), index=False)